In [1]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path
import os

import argparse

In [2]:
exp_id = 'eff_f30_e5_s10000'

In [3]:
# initialize
module_path = f'experiment.{exp_id}'
exec(f'from {module_path}.config import *', globals()) 
exec(f'from {module_path}.model import *', globals())

In [26]:
import os
import random
import time
import argparse
from tqdm import tqdm
from tempfile import gettempdir

import numpy as np
import pandas as pd
import torch
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader
from ranger import Ranger  # this is from ranger.py
from ranger import RangerVA  # this is from ranger913A.py
from ranger import RangerQH  # this is from rangerqh.py


from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from pathlib import Path


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-e', '--exp_id', type=str, default='', required=True,
                        help='path of your experiment directory name')
    return parser.parse_args()


def initialize(exp_id):

    # seed = int(time.time())
    seed = int(time.time())
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    np.set_printoptions(precision=4)
    # load experiment config and model architecture
    module_path = f'experiment.{exp_id}'
    exec(f'from {module_path}.config import *', globals())
    exec(f'from {module_path}.model import LyftModel', globals())
    exec(f'from {module_path}.model import forward', globals())
    try:
        exec(f'from {module_path}.model import load_pretrained', globals())
    except Exception as e:
        print(f"There is no load_pretrained function in {module_path}.")
    print('GPU =', GPU)


def get_dm():
    # set env variable for data
    DIR_INPUT = cfg["data_path"]
    os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
    dm = LocalDataManager(None)
    return dm


def get_device():
    gpu = None
    if not ('GPU' in globals()):
        gpu = True
    else:
        gpu = GPU

    print('GPU =', gpu)

    if gpu is False:
        return "cpu"
    else:
        return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def load_train_data():
    # load training data
    dm = get_dm()
    train_cfg = cfg["train_data_loader"]

    rasterizer = build_rasterizer(cfg, dm)
    train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
    return train_dataset
#     train_dataloader = DataLoader(train_dataset,
#                                   shuffle=train_cfg["shuffle"],
#                                   batch_size=train_cfg["batch_size"],
#                                   num_workers=train_cfg["num_workers"])
#     print('len(train_dataloader):', len(train_dataloader))
#     return train_dataloader


def load_test_data():
    dm = get_dm()

    # print(eval_dataset)
    test_cfg = cfg["test_data_loader"]
    rasterizer = build_rasterizer(cfg, dm)
    test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
    test_mask = np.load(f'{cfg["data_path"]}/scenes/mask.npz')["arr_0"]
    test_dataset = AgentDataset(
        cfg, test_zarr, rasterizer, agents_mask=test_mask)
    
    
    test_dataloader = DataLoader(test_dataset,
                                 shuffle=test_cfg["shuffle"],
                                 batch_size=test_cfg["batch_size"],
                                 num_workers=test_cfg["num_workers"])
    return test_dataloader


def load_train_data():
    dm = get_dm()

    train_cfg = cfg["train_data_loader"]


    train_base_path = '/home/axot/lyft/data/scenes/train_chopped_100'

    train_zarr_path = str(Path(train_base_path) /
                         Path(dm.require(train_cfg["key"])).name)

    rasterizer = build_rasterizer(cfg, dm)
    train_zarr = ChunkedDataset(train_zarr_path).open()
    
    # ===== INIT DATASET AND LOAD MASK
    train_dataset = AgentDataset(
        cfg, train_zarr, rasterizer, )

    return train_dataset

In [23]:
dataset = load_val_data()

In [24]:
len(dataset)

2270313

In [27]:
total_train = load_train_data()
total_train

In [28]:
len(total_train)

22496709

In [21]:
dataset[0]['history_positions']

array([[ 4.5474735e-13,  0.0000000e+00],
       [-8.3277971e-01, -2.0348351e-02],
       [-1.6913595e+00, -5.9234224e-02],
       [-2.5541222e+00, -8.9041762e-02],
       [-3.4227960e+00, -1.0623392e-01],
       [-4.2846088e+00, -1.5802377e-01],
       [-6.0346489e+00, -3.3230996e-01],
       [-7.6491122e+00, -3.6506152e-01],
       [-8.8082504e+00, -3.9701042e-01],
       [-9.3324175e+00, -4.3215683e-01],
       [-9.4165421e+00, -4.3820789e-01],
       [-1.2629728e+01, -1.9397324e-01],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00],
       [ 0.00000

In [25]:
device='cuda:0'

In [24]:
agent_indices = eval_dataset.get_frame_indices(599) 
agent_indices

array([4, 5])

In [25]:
data_agent = eval_dataset[4]

In [37]:
data_agent.keys()

dict_keys(['image', 'target_positions', 'target_yaws', 'target_availabilities', 'history_positions', 'history_yaws', 'history_availabilities', 'world_to_image', 'raster_from_world', 'raster_from_agent', 'agent_from_world', 'world_from_agent', 'track_id', 'timestamp', 'centroid', 'yaw', 'extent'])

In [39]:
data_agent['history_positions'][0] 

array([0.0000000e+00, 1.1368684e-13], dtype=float32)

In [58]:
np.tile([data_agent["history_positions"][0]], (len(out_pos), 1) ,)

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.

In [56]:
data_agent["history_positions"]

array([[ 0.0000000e+00,  0.0000000e+00],
       [-1.4693557e-03, -8.7270926e-04],
       [-6.5898284e-04, -2.2371109e-03],
       [-5.9982651e-04, -3.0538440e-03],
       [-7.7665702e-04, -5.1465635e-03],
       [-7.8683562e-04, -2.7389820e-03],
       [-1.3217841e-03, -6.4308255e-04],
       [-4.5893426e-04, -1.9763161e-03],
       [-1.7969140e-04, -5.0759576e-03],
       [ 5.1936740e-04, -7.9262536e-03],
       [ 4.4213615e-05, -1.1190006e-02]], dtype=float32)